# Predicting stocks using technical indicators (atr, lwma)
This template shows you the basic steps for taking part to the NASDAQ-100 Stock Long-Short contest.

In [ ]:
import xarray as xr

import qnt.ta as qnta
import qnt.data as qndata
import qnt.output as qnout
import qnt.stats as qns
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def strategy2(data, wma, limit):
    vol = data.sel(field="vol")
    liq = data.sel(field="is_liquid")
    close = data.sel(field="close")
    high = data.sel(field="high")
    low = data.sel(field="low")

    atr = qnta.atr(high=high, low=low, close=close, ma=14)
    ratio = atr / close
    weights = xr.where(ratio > limit, 0, 1)

    money_vol = vol * liq * close
    total_money_vol = money_vol.sum(dim='asset')
    money_vol_share = money_vol / total_money_vol

    return qnta.lwma(money_vol_share, wma) * weights


data = qndata.stocks.load_ndx_data(min_date="2005-01-01")
weights_1 = strategy2(data, wma=135, limit=0.0205)

In [ ]:
def get_enough_bid_for(weights_):
    time_traded = weights_.time[abs(weights_).fillna(0).sum('asset') > 0]
    is_strategy_traded = len(time_traded)
    if is_strategy_traded:
        return xr.where(weights_.time < time_traded.min(), data.sel(field="is_liquid"), weights_)
    return weights_

weights_new = get_enough_bid_for(weights_1)
weights_new = weights_new.sel(time=slice("2006-01-01", None))

weights = qnout.clean(output=weights_new, data=data, kind="stocks_nasdaq100")

In [ ]:
def print_statistic(data, weights_all):
    import qnt.stats as qnstats

    stats = qnstats.calc_stat(data, weights_all)
    display(stats.to_pandas().tail(5))
    # graph
    performance = stats.to_pandas()["equity"]
    import qnt.graph as qngraph

    qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

print_statistic(data, weights)

In [ ]:
qnout.check(weights, data, "stocks_nasdaq100")
qnout.write(weights)  # to participate in the competition

In [ ]:
weights

In [ ]:
data